In [ ]:
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from research.weight_estimation.keypoint_utils.keypoint_transformations import get_keypoint_arr

In [ ]:
df = pd.read_csv('/root/data/s3/aquabyte-images-adhoc/alok/production_datasets/imr_data_one_month.csv')
df1 = pd.read_csv('/root/data/temp/results_current_pytorch_imr.csv')
df2 = pd.read_csv('/root/data/temp/results_new_biomass_keras_model_imr_v2.csv')


In [ ]:
raw_keypoint_arrs, centered_keypoint_arrs = [], []
for idx, row in df.iterrows():
    raw_keypoint_arr, centered_keypoint_arr = get_keypoint_arr(json.loads(row.annotation), json.loads(row.camera_metadata))
    raw_keypoint_arrs.append(raw_keypoint_arr)
    centered_keypoint_arrs.append(centered_keypoint_arr)



In [ ]:
df['raw_keypoint_arr'] = raw_keypoint_arrs
df['centered_keypoint_arr'] = centered_keypoint_arrs

In [ ]:
raw_depths = []
centered_depths = []
for idx, row in df.iterrows():
    raw_keypoint_arr, centered_keypoint_arr = row.raw_keypoint_arr, row.centered_keypoint_arr
    raw_median_depth = np.median(raw_keypoint_arr[:, 1])
    centered_median_depth = np.median(centered_keypoint_arr[:, 1])
    raw_depths.append(raw_median_depth)
    centered_depths.append(centered_median_depth)
    
df['raw_depth'] = raw_depths
df['centered_depth'] = centered_depths
df['difference'] = df2.estimated_weight_g - df1.estimated_weight_g

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(df.raw_depth)
plt.grid()
plt.show()

In [ ]:
vals = np.arange(0.5, 3.0, 0.1)
for idx in range(len(vals) - 1):
    low, high = vals[idx], vals[idx+1]
    mean_diff = df[(df.centered_depth > low) & (df.centered_depth < high)].difference.mean()
    print('Mean Difference for depth range {}<->{}: {}'.format(round(low, 1), round(high, 1), mean_diff))

In [ ]:
json.loads(df[df.right_crop_url == 'https://aquabyte-crops.s3.eu-west-1.amazonaws.com/environment=production/site-id=40/pen-id=61/date=2019-11-23/hour=00/at=2019-11-23T00:31:20.795673000Z/right_frame_crop_1050_24_3358_837.jpg'].annotation.iloc[0])